In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
valid_filenames

['000000.ogg',
 '000001.ogg',
 '000002.ogg',
 '000004.ogg',
 '000005.ogg',
 '000007.ogg',
 '000009.ogg',
 '000010.ogg',
 '000012.ogg',
 '000013.ogg',
 '000014.ogg',
 '000015.ogg',
 '000016.ogg',
 '000018.ogg',
 '000019.ogg',
 '000020.ogg',
 '000021.ogg',
 '000022.ogg',
 '000025.ogg',
 '000026.ogg',
 '000027.ogg',
 '000028.ogg',
 '000029.ogg',
 '000030.ogg',
 '000033.ogg',
 '000034.ogg',
 '000035.ogg',
 '000037.ogg',
 '000039.ogg',
 '000040.ogg',
 '000041.ogg',
 '000042.ogg',
 '000043.ogg',
 '000045.ogg',
 '000046.ogg',
 '000047.ogg',
 '000049.ogg',
 '000051.ogg',
 '000052.ogg',
 '000053.ogg',
 '000054.ogg',
 '000055.ogg',
 '000056.ogg',
 '000058.ogg',
 '000059.ogg',
 '000061.ogg',
 '000062.ogg',
 '000063.ogg',
 '000066.ogg',
 '000068.ogg',
 '000069.ogg',
 '000070.ogg',
 '000071.ogg',
 '000072.ogg',
 '000073.ogg',
 '000074.ogg',
 '000075.ogg',
 '000076.ogg',
 '000077.ogg',
 '000078.ogg',
 '000080.ogg',
 '000081.ogg',
 '000082.ogg',
 '000083.ogg',
 '000084.ogg',
 '000085.ogg',
 '000087.o

In [10]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [11]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [12]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [13]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [14]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [15]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [16]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

### Create Folds

In [17]:
df_train['fold'] = 0

In [18]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [19]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [20]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [21]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((128,128))])
    [torchvision.transforms.CenterCrop((128,960))])


In [22]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [23]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/{filename}')
        else:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/{filename}')
            
        audio = np.mean(audio, axis=0, keepdims=True)   ### convert to mono
        return audio

    def resample(self,audio,sr):
        
        audio = torchaudio.transforms.Resample(orig_freq= sr, new_freq=  16000, 
                                    resampling_method = 'sinc_interpolation', 
                                    lowpass_filter_width = 6, 
                                    rolloff = 0.99)(audio)
        return audio

    def makeCQT(self,audio,sr):
        audio = CQT1992v2(hop_length=self.HOP_LEN,n_bins=self.n_bins,bins_per_octave=self.bins_per_octave,
                          sr=sr,verbose=False)(audio)
        
        max_ = audio.max()
        if max_ < 0.001:
            audio[...] = 0
        else:
            audio /= max_
        
        return audio
        
        
    def create_spectrogram(self,audio,sr):
                
        specgram = torchaudio.transforms.MelSpectrogram(sample_rate=sr,
                                                        n_fft=self.N_FFT, 
                                                        win_length=self.N_FFT, 
                                                        hop_length=self.HOP_LEN,
                                                        center=True,
                                                        pad_mode="reflect",
                                                        power=2.0,
                                                        norm='slaney',
                                                        onesided=True,
                                                        n_mels=self.n_mels,
                                                        mel_scale="htk"
                                                       )(audio)

        specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
        specgram = specgram - specgram.min()
        
        max_ = specgram.max()
        if max_ < 0.001:
            specgram[...] = 0
        else:
            specgram /= max_
        return specgram

    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        sr = 16000
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
#         samples = self.load_file(row.filename)
    
        samples = np.stack(samples, axis=0)
        
        
        samples = torch.from_numpy(samples).float().view(1,-1)
        
        if self.audiotfms is not None:
             samples = self.audiotfms(samples.unsqueeze(0)).squeeze(0)

        samples = self.create_spectrogram(samples,sr)
#         samples = self.makeCQT(samples,sr)

        if self.transform is not None:
            samples = self.transform(samples)

            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [24]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [25]:
a,b = next(iter(POGFiles(df_train,transform=tfms,n_mels=128,HOP_LEN=512,N_FFT=512)))
#` a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=576,N_FFT=512)))
a.shape

torch.Size([1, 128, 960])

In [26]:
def gem_1d_res(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), 2,2).pow(1./p)

def gem_1d(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), (x.size(-1),)).pow(1./p )

#### FastAI model

In [27]:
df_train['genre_id'].nunique()

19

In [28]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [29]:
import timm
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_x

In [30]:
timm.create_model('tf_efficientnet_b1',in_chans=1)

EfficientNet(
  (conv_stem): Conv2dSame(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
      (

In [31]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [32]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [33]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'tf_efficientnet_b1_MonoSpec_Hop512_Mels128'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 36
        n_epochs = 7

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                            )

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('tf_efficientnet_b1', pretrained=False, in_chans=1)#get_model(resnet34)
        model.conv_stem.stride = (1,1)
#         model.stem[0].stride = (2,2)
        model.classifier = nn.Linear(1280,19)
        f1_score = F1Score(average="micro")
        
        ### MixUp() in callbacks
#        ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1,patience=2,min_lr=1e-7)
# GradientAccumulation (n_acc = 2)

        learn = Learner(dls, model,opt_func=ranger,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
          
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
#         learn.fit(n_epochs, 1e-2)       
#         learn.fit_one_cycle(n_epochs, 1e-2, wd=1e-03)
        learn.fit_flat_cos(n_epochs, 1e-2, wd=1e-03)

        learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')

        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           n_mels=128,HOP_LEN=512,N_FFT=512,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [34]:
df_test_valid.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


In [35]:
df_test_valid

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg
...,...,...,...
5071,6427,006427.ogg,test/006427.ogg
5072,16903,016903.ogg,test/016903.ogg
5073,1731,001731.ogg,test/001731.ogg
5074,12871,012871.ogg,test/012871.ogg


In [36]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.894710,2.966105,0.229282,10:19
1,1.736675,1.818141,0.365143,10:25
2,1.641828,1.651392,0.447012,10:23
3,1.585770,1.609616,0.463084,10:25
4,1.535589,1.916248,0.395530,10:25
5,1.456177,1.642688,0.472627,10:24
6,1.314228,1.405222,0.537670,10:23


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.866414,1.937334,0.363887,10:05
1,1.721495,1.711381,0.433199,10:30
2,1.627840,1.622545,0.460321,10:25
3,1.552991,1.555400,0.496735,10:25
4,1.510753,1.738393,0.442491,10:26
5,1.441162,1.515410,0.512054,10:25
6,1.307964,1.381107,0.543194,10:26


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.868991,1.941743,0.368910,10:06
1,1.709766,1.674041,0.439227,10:51
2,1.630355,1.674832,0.433702,10:25
3,1.569664,1.611660,0.455299,10:25
4,1.531142,1.675803,0.435711,10:24
5,1.452547,1.555261,0.475892,10:25
6,1.338387,1.425409,0.525113,10:24


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.897922,2.255773,0.286611,10:19
1,1.712824,1.920893,0.380809,10:27
2,1.615941,1.583438,0.464456,10:26
3,1.542974,1.555472,0.480281,10:26
4,1.500840,1.541937,0.488320,10:25
5,1.421206,1.536744,0.480281,10:26
6,1.297238,1.404152,0.538558,10:25


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.861281,2.069498,0.342627,10:06
1,1.711122,1.823617,0.402160,10:26
2,1.615438,1.734346,0.434815,10:26
3,1.539937,1.592660,0.474253,10:25
4,1.484112,1.761413,0.442602,10:26
5,1.415224,1.580861,0.472243,10:25
6,1.275422,1.428486,0.535544,10:25


In [37]:
submission

,song_id,genre_id
0,7072,0
1,10207,0
2,20008,0
3,10924,0
4,21896,0
...,...,...
5073,6427,0
5074,16903,0
5075,1731,0
5076,12871,0


### OOF

In [38]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'tf_efficientnet_b1_MonoSpec_Hop512_Mels128'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [39]:
OOFOut['ID']

0        10150
1        11198
2        13166
3        12275
4        24102
         ...  
19903     2609
19904    13023
19905     2022
19906     9337
19907     6579
Name: ID, Length: 19908, dtype: int64

In [40]:
OOFOut.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
19903,0.299320,0.012698,0.002831,0.006230,0.513632,0.002062,0.015600,0.006781,0.041633,0.050479,0.017703,0.000205,4.459436e-06,0.003477,0.000168,0.021910,0.000029,0.001215,4.022091e-03,2609
19904,0.025919,0.097006,0.025117,0.112323,0.003063,0.328528,0.005629,0.023628,0.116899,0.115042,0.001943,0.007070,8.652142e-06,0.080580,0.028541,0.010467,0.004930,0.012866,4.421237e-04,13023
19905,0.011936,0.634452,0.271772,0.006407,0.000056,0.009460,0.002065,0.010678,0.044354,0.000256,0.001606,0.000004,6.136139e-09,0.000740,0.005771,0.000156,0.000001,0.000286,3.559845e-08,2022
19906,0.667355,0.003345,0.000832,0.011079,0.150109,0.000026,0.054316,0.003987,0.004210,0.002269,0.101738,0.000004,1.923830e-06,0.000251,0.000002,0.000208,0.000002,0.000045,2.197232e-04,9337
19907,0.348403,0.022514,0.001488,0.068061,0.012899,0.013071,0.022202,0.212168,0.039217,0.011627,0.218292,0.009081,2.946709e-05,0.014201,0.000139,0.003597,0.000089,0.001131,1.792655e-03,6579


In [41]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [42]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [43]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

0.5356138235885072

In [44]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
row_0,,,,,,,,,,,,,,,,,,,
0,2144,190,50,213,358,26,442,155,194,143,479,4,0,30,1,42,1,1,6
1,130,1615,884,175,20,166,13,120,285,69,17,6,0,48,61,13,2,14,0
2,23,778,1486,42,16,16,2,22,77,14,0,0,0,3,13,4,3,7,0
3,195,171,66,970,24,50,39,176,76,42,112,40,16,47,3,2,39,4,2
4,173,19,18,32,1282,6,12,2,28,28,12,0,0,1,0,6,5,5,3
5,25,145,23,80,5,798,1,52,126,61,2,18,0,30,31,5,8,10,0
6,193,11,10,43,14,3,653,6,12,5,24,0,0,4,0,1,0,0,0
7,81,65,6,99,5,47,5,414,27,2,92,53,1,34,0,2,0,3,1
8,23,23,8,9,8,17,0,5,50,12,3,0,0,4,1,5,0,2,0


In [ ]:
df = OOFOut.groupby('genre_id').size()
df = 1. / df
df = df / df.mean()

class_weights = df.values
logits_weights = np.log((class_weights).reshape((1, -1)))
probit_weights = scipy.special.softmax(logits_weights,1)

In [ ]:
probit_weights

In [ ]:
f1_score(OOFOut['genre_id'],OOFOut.iloc[:,:19].idxmax(1),average="micro")

In [ ]:
for a in [13,14,15,16]:
    print(a, OOFOut[OOFOut['genre_id'].isin([a])][a].mean(),OOFOut[~OOFOut['genre_id'].isin([a])][a].mean())

### Mode based sub

In [45]:
kernel_type

'tf_efficientnet_b1_MonoSpec_Hop512_Mels128'

In [46]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [47]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub

In [52]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['tf_efficientnet_b1_MonoSpec_Hop512_Mels128','tf_efficientnet_b1_ns_MonoSpec_Hop576_Mels128',
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [53]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2models.csv',index=False)

In [54]:
out

,song_id,target_x,target_y,target_x,target_y,target_x,target_y,target_x,target_y,target_x,target_y
0,7072,7.0,1.0,1.0,3.0,1.0,6.0,1.0,1.0,0.0,0.0
1,10207,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0
2,20008,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0
3,10924,0.0,9.0,9.0,9.0,9.0,9.0,0.0,6.0,6.0,0.0
4,21896,4.0,4.0,4.0,4.0,2.0,9.0,2.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
5073,6427,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5074,16903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5075,1731,3.0,6.0,3.0,3.0,3.0,3.0,3.0,6.0,3.0,6.0
5076,12871,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### End ###